# Titanic Model Deployment

The purpose of this notebook is to demonstrate how the model yielded by `titanic-ml.ipynb` can be deployed as a RESTful service on Kubernetes.

Required steps:

1. inject model into a Flask service:
    - create 'deploy' directory;
    - 'download' model locally;
    - 'download' Flask model wrapper;
    - use environment variable to pass model name to Flask service;
2. inject Flask service into Docker container:
    - build Docker image;
    - push image to registry;
3. deploy end-to-end service to Kubernetes using a Helm chart:
    - download the appropriate Helm chart;
    - set the appropriate parameter values for service naming, clusters, etc.; and,
    - deploy using Helm!

In [27]:
import docker

## Mandatory Deployment Parameters

In [23]:
MODEL = 'models/titanic-ml-2019-02-11T17:48:28.joblib'
SERVICE_NAME = 'titanic'
API_VERSION = '1'
DOCKER_IMAGE_REGISTRY = 'alexioannides'

## Copy Model to `py-flask-ml-service`

In [57]:
_ = os.popen(f'cp {MODEL} deploy/py-flask-ml-service/model.joblib')

## Copy `Pipfile` ?

- how to best manage the needs of the service with the needs of the notebook?

In [40]:
# TODO

## Write Service Parameters to `.env` File in `py-flask-ml-service`

In [33]:
with open('deploy/py-flask-ml-service/.env', 'w') as file:
    file.writelines(f'SERVICE_NAME={SERVICE_NAME}\n')
    file.writelines(f'API_VERSION={API_VERSION}\n')

## Build Docker Image

- Note, we will need a build server (e.g. Travis) to build and push.

In [64]:
image_tag = f'{DOCKER_IMAGE_REGISTRY}/{SERVICE_NAME}:latest'

docker_client = docker.from_env()
image, build_log = docker_client.images.build(
    path='deploy/py-flask-ml-service', tag=image_tag, rm=True)

## Push Image to DockerHub

- Will need to use `docker_client.login()` on a build server.

In [76]:
_ = docker_client.images.push(image_tag)

## Deploy to Kubernetes using Helm Charts

In [97]:
!helm install deploy/helm-ml-prediction-app \
    --set app.name="$SERVICE_NAME-survival-prediction" \
    --set app.namespace="$SERVICE_NAME" \
    --set app.image="$image_tag"

NAME:   xrayed-olm
LAST DEPLOYED: Wed Feb 13 00:11:58 2019
NAMESPACE: default
STATUS: DEPLOYED

RESOURCES:
==> v1/Namespace
NAME     STATUS  AGE
titanic  Active  0s

==> v1/Service
NAME                            TYPE          CLUSTER-IP     EXTERNAL-IP  PORT(S)         AGE
titanic-survival-prediction-lb  LoadBalancer  10.98.211.202  <pending>    5000:30438/TCP  0s

==> v1/ReplicationController
NAME                            DESIRED  CURRENT  READY  AGE
titanic-survival-prediction-rc  2        2        0      0s

==> v1/Pod(related)
NAME                                  READY  STATUS             RESTARTS  AGE
titanic-survival-prediction-rc-bsr8f  0/1    ContainerCreating  0         0s
titanic-survival-prediction-rc-x74ph  0/1    ContainerCreating  0         0s


NOTES:
Thank you for installing helm-ml-score-app.

Your release is named xrayed-olm.

To learn more about the release, try:

  $ helm status xrayed-olm
  $ helm get xrayed-olm



## Test Prediction Service

We will assume that Minikube is being used for local testing, in which case we will need to ask it for the URL of its 'virtual load balancer' used for our service.

In [98]:
!minikube service list

|-------------|--------------------------------|-----------------------------|
|  NAMESPACE  |              NAME              |             URL             |
|-------------|--------------------------------|-----------------------------|
| default     | kubernetes                     | No node port                |
| kube-system | kube-dns                       | No node port                |
| kube-system | kubernetes-dashboard           | No node port                |
| kube-system | tiller-deploy                  | No node port                |
| titanic     | titanic-survival-prediction-lb | http://192.168.99.114:30438 |
|-------------|--------------------------------|-----------------------------|


We can then take the appropriate URL from the above and test our titanic prediction service!

In [100]:
!curl http://192.168.99.114:30438/titanic/v1/predict \
        --request POST\
        --header 'Content-Type: application/json' \
        --data '{"Pclass": [1], "Sex": ["male"], "Age": [32], "SibSp": [1],                      "Parch": [0], "Fare": [100], "Embarked": ["S"]}'

{"prediction":[1]}
